In [1]:
import os
import openai
import tiktoken
from IPython.display import display, Markdown, Latex, HTML, JSON

MODEL_NAME = 'gpt-3.5-turbo'

with open('/home/loc/Documents/OPENAI_API_KEY.txt') as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    openai.api_key = OPENAI_API_KEY

def get_completion(prompt, model=MODEL_NAME, temperature=0):
    # generate message
    messages = [{'role':'user',
                 'content':prompt}
               ]
    # receive the response
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature # this is the degree of randomness of model
    )
    return response.choices[0].message['content']


def get_completion_from_messages(messages,model=MODEL_NAME,
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature, # this is the degree of randomness of model
        max_tokens = max_tokens # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message['content']

def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

### Classify customer queries to handle different cases

In [2]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.

Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human
"""


user_message = f"""\
I want you to delete my profile and all of my user data"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

{
  "primary": "Account Management",
  "secondary": "Close account"
}


In [4]:
user_message = f"""\
I want to know how to check my payment"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

{
    "primary": "Billing",
    "secondary": "Add a payment method"
}


In [5]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "General Inquiry",
    "secondary": "Product information"
}
